In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
from ase.io import read, write
from ase.visualize import view
from ase.io import read
from ase.db import connect
import dask
import dask.threaded

from ase.calculators.singlepoint import SinglePointCalculator
%pylab inline
from ase.db import connect

Populating the interactive namespace from numpy and matplotlib


/home/sebastian/miniconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['connect']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
# train_molecules = pd.read_csv('./data/structures.csv').groupby('molecule_name').groups.keys()

In [15]:
@dask.delayed    
def add_data(idx, args):
    try:
        molecule_name, idx0, idx1, coupling = args
    except ValueError:
        molecule_name, idx0, idx1 = args
        coupling = 0
        
    path = 'data/structures/' + molecule_name  + '.xyz'
    atoms = read(path)
    atom_mask = np.zeros(len(atoms.positions))
    atom_mask[idx0] = 1
    atom_mask[idx1] = 1
    entry = dict(atoms = atoms, _atom_mask = atom_mask, coupling = coupling, champs_id = idx)
    
    return entry

In [16]:
train = pd.read_csv('./data/test.csv')

for d,el in [(1,'N'),(2, 'N'),(3, 'N'),(1,'C'),(2,'C'),(3,'C'),(2,'H'),(3,'H')]:
    type_sel = train[train['type']=='{}JH{}'.format(d,el)].set_index('id',drop=True).drop(['type'],axis=1)

    step_size = int(np.ceil(len(type_sel)/100))

    for i in range(100):

        futures = [add_data(idx, args) for idx, args in  type_sel.iloc[i*step_size:(i+1)*step_size].iterrows()]

        results = dask.compute(*futures, scheduler='threads')

        with connect('data/jh{}_{}.db'.format(el.lower(),d)) as db:
            for idx, entry in enumerate(results):
                mask = entry.pop('_atom_mask')
                db.write(**entry, data ={'_atom_mask': mask})
        del results

In [2]:
train = pd.read_csv('lgbm_train_')

In [3]:
jhn1 = train[train['type'] == '1JHN']

total_len = len(jhn1)

jhn1['diff'] = jhn1['scalar_coupling_constant'] - jhn1['scalar_coupling_constant_predicted']




ids = []
coupling = []
for idx, (item, (_, row)) in enumerate(zip(db.select(),jhn1.iterrows())):
    if idx%int(total_len/100) == 0:
        print('{} %'.format(int(idx/(total_len/100) )))
    
    coupling.append(row['diff'])
    ids.append(item.id)
    if idx%100 == 0:
        with connect('data/training/jhn_1_lgbm.db') as db:
            for i, c in zip(ids,coupling):
#                 print(i,c)
                db.update(i, coupling_min_lgbm = c)
        coupling = []
        ids = []
        
with connect('data/training/jhn_1.db') as db:
    for i, c in zip(ids,coupling):
    #                 print(i,c)
        db.update(i, coupling_min_lgbm = c)
   
    

In [19]:
with connect('data/training/jhn_1_lgbm.db') as db:
    for i, c in zip(ids,coupling):
    #                 print(i,c)
        db.update(i, coupling_min_lgbm = c)
   
    

In [19]:
item.ida

12

In [32]:
a, b = jhn1.iterrows().__next__()

In [34]:
b['scalar_coupling_constant'] - b['scalar_coupling_constant_predicted']

-0.002361291503909513